In [1]:
# %load insurance_simple_linear_regression_gd_tf_test.py
import tensorflow as tf
import math
import numpy as np
import pandas as pd

## 데이터 수집

insurance = pd.read_csv("insurance.csv")
print(insurance[0:5])
print(type(insurance))

age = insurance["age"].values
expenses = insurance["expenses"].values
print(age, type(age))
print(expenses, type(expenses))

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


   age     sex   bmi  children smoker     region  expenses
0   19  female  27.9         0    yes  southwest  16884.92
1   18    male  33.8         1     no  southeast   1725.55
2   28    male  33.0         3     no  southeast   4449.46
3   33    male  22.7         0     no  northwest  21984.47
4   32    male  28.9         0     no  northwest   3866.86
<class 'pandas.core.frame.DataFrame'>
[19 18 28 ... 18 21 61] <class 'numpy.ndarray'>
[16884.92  1725.55  4449.46 ...  1629.83  2007.95 29141.36] <class 'numpy.ndarray'>


In [2]:
## 예측 모델 정의

X = tf.placeholder("float")
Y = tf.placeholder("float")

tf_coef = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
tf_intercept = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
tf_expenses_pred = tf_coef * X + tf_intercept

In [3]:
## 비용 함수, 최적화 함수 정의

tf_cost = tf.reduce_mean(tf.square(tf_expenses_pred - Y))
a = tf.Variable(0.1) # 학습률 alpha. 가중치가 infinity, nan으로 발산
#a = tf.Variable(0.0001) # 학습률 alpha
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(tf_cost)
# 변수 초기화
init = tf.global_variables_initializer()

In [4]:
## 훈련
sess = tf.Session()
sess.run(init)
for step in range(30000):
    sess.run(train, feed_dict={X: age, Y: expenses})
    if step % 10000 == 0:
        cost, coef, intercept = sess.run([tf_cost, tf_coef, tf_intercept],
                                         feed_dict={X: age, Y: expenses})
        print(step, cost, math.sqrt(cost), coef, intercept)
sess.close()

0 22499322000000.0 4743345.07441489 [114163.945] [2652.651]
10000 nan nan [nan] [nan]
20000 nan nan [nan] [nan]


In [5]:
## 재훈련 (발산 원인 분석)
sess = tf.Session()
sess.run(init)
init = tf.global_variables_initializer()
for step in range(10):
    sess.run(train, feed_dict={X: age, Y: expenses})
    if step % 1 == 0:
        cost, coef, intercept = sess.run([tf_cost, tf_coef, tf_intercept],
                                         feed_dict={X: age, Y: expenses})
        print(step, cost, math.sqrt(cost), coef, intercept)
sess.close()

0 22530247000000.0 4746603.7435589675 [114242.16] [2654.244]
1 2.6982463e+18 1642633963.992324 [-39421680.] [-891041.5]
2 3.231469e+23 568460106031.2906 [1.3642623e+10] [3.0841117e+08]
3 3.870066e+28 196724832904659.88 [-4.7212507e+12] [-1.0673061e+11]
4 4.6348627e+33 6.807982573310499e+16 [1.6338656e+15] [3.6935855e+13]
5 inf inf [-5.6542564e+17] [-1.2782253e+16]
6 inf inf [1.956747e+20] [4.4235057e+18]
7 inf inf [-6.7716403e+22] [-1.5308257e+21]
8 inf inf [2.343436e+25] [5.2976707e+23]
9 inf inf [-8.109841e+27] [-1.833345e+26]


In [6]:
## 재훈련 (학습률 조정)
sess = tf.Session()
sess.run(init)
assign = a.assign(0.0001)
sess.run(assign)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(tf_cost)

for step in range(300000):
    sess.run(train, feed_dict={X: age, Y: expenses})
    if step % 10000 == 0:
        cost, coef, intercept = sess.run([tf_cost, tf_coef, tf_intercept],
                                         feed_dict={X: age, Y: expenses})
        print(step, cost, math.sqrt(cost), coef, intercept)

0 214883500.0 14658.905279726723 [114.062386] [2.2657852]
10000 134161130.0 11582.794481471214 [314.60992] [649.46375]
20000 133898030.0 11571.431717812624 [303.0418] [1161.1812]
30000 133731050.0 11564.214110781588 [293.82608] [1568.8401]
40000 133625064.0 11559.630789951727 [286.48447] [1893.5985]
50000 133557816.0 11556.721680476692 [280.63562] [2152.3245]
60000 133515110.0 11554.873949983185 [275.9764] [2358.4243]
70000 133488040.0 11553.702436881435 [272.26465] [2522.6152]
80000 133470850.0 11552.958408996372 [269.3075] [2653.4255]
90000 133459950.0 11552.486831847073 [266.95206] [2757.6184]
100000 133453016.0 11552.18663284142 [265.07516] [2840.6433]
110000 133448620.0 11551.996537395604 [263.58054] [2906.7598]
120000 133445816.0 11551.874999323703 [262.38947] [2959.4468]
130000 133444056.0 11551.798820962907 [261.4404] [3001.427]
140000 133442930.0 11551.749997294783 [260.68527] [3034.8323]
150000 133442216.0 11551.719179412214 [260.0828] [3061.4817]
160000 133441750.0 11551.699

In [7]:
## 예측
coef = sess.run(tf_coef)
intercept = sess.run(tf_intercept)
print(coef * np.array([20, 40, 60]) + intercept)

print(sess.run(tf_expenses_pred, feed_dict={X: [20.0, 40.0, 60.0]}))
sess.close()

[ 8317.39013672 13474.27612305 18631.16210938]
[ 8317.39  13474.275 18631.162]
